```
pip install -U torch
pip install -U click
pip install -U 'protobuf==3.20.*'
pip install wandb
wandb login e5292edda95a11630042fdf943d60d2bbf749fcf
pip install datasets
pip install tokenizers
pip install transformers
```

In [4]:
# pyt
import torch as t
import torch.nn as nn
from torch.utils.data import DataLoader

# data pipeline
from datasets import load_dataset, DatasetDict, load_from_disk
from typing import cast
import math, random

# tokenization
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.processors import TemplateProcessing
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.trainers import BpeTrainer
from transformers import PreTrainedTokenizerFast

# logging
import os, argparse
import wandb


/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [5]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [6]:
hyper = {
    'vs': 2**13,
    'ly': 4,
    'hs': 768,
    'ah': 4,
    'cx': 512,
    'lr': 1e-4,
    'bs': 256,
    'ac': 4,
    'ep': 10,
}

hyper = argparse.Namespace(**hyper)


In [7]:
dataset = cast(DatasetDict, load_dataset('skeskinen/TinyStories-Instruct-hf'))
dataset['train'].set_format(type='torch', columns=['text'])
dataset['train'].format['type']
dataset['validation'].set_format(type='torch', columns=['text'])
dataset['validation'].format['type']
print(dataset)

Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/skeskinen___parquet/skeskinen--TinyStories-Instruct-hf-1f9111cb77858404/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2476533
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 25028
    })
})


In [8]:
tok = Tokenizer(BPE())
tok.normalizer = Lowercase()
tok.pre_tokenizer = ByteLevel()
tok.decoder = ByteLevelDecoder()
tok.post_processor = TemplateProcessing(single='$0 <|endoftext|>', special_tokens=[('<|endoftext|>', 1)],)
tok.enable_truncation(max_length=hyper.cx)
tok.enable_padding(pad_token='<pad>', length=hyper.cx)
trainer = BpeTrainer(vocab_size=hyper.vs, initial_alphabet=ByteLevel.alphabet(), special_tokens=['<pad>', '<|endoftext|>', '\n','Words: ', 'Features: ', 'Random sentence: ', 'Summary: ', 'Story: '])

In [9]:
if os.path.isfile('tiny.json'): tok = Tokenizer.from_file('tiny.json')
else: tok.train_from_iterator(dataset['train']['text'], trainer=trainer); tok.save('tiny.json')

tok = PreTrainedTokenizerFast(tokenizer_object=tok)
tok.pad_token = 0


In [10]:
def tokenization(example):
    return tok(example['text'], truncation=True, max_length=hyper.cx, padding='max_length')

if os.path.exists('train_dataset') and os.path.exists('valid_dataset'):
    train = load_from_disk('train_dataset')
    valid = load_from_disk('valid_dataset')
else:
    train = dataset['train'].map(tokenization, batched=True, batch_size=8192, writer_batch_size=8192)
    valid = dataset['validation'].map(tokenization, batched=True, batch_size=8192, writer_batch_size=8192)
    train.save_to_disk('train_dataset')
    valid.save_to_disk('valid_dataset')


In [11]:
train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])
train.format['type']
valid.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])
valid.format['type']

'torch'

In [12]:
trainl = DataLoader(train, batch_size=hyper.bs, shuffle=True, drop_last=True)
validl = DataLoader(valid, batch_size=hyper.bs, shuffle=True, drop_last=True)

In [13]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len, dropout = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = t.arange(max_len).unsqueeze(1)
        div_term = t.exp(t.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = t.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = t.sin(position * div_term)
        pe[:, 0, 1::2] = t.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
    
class trans(nn.Module):
    def __init__(self):
        super().__init__()
        self.inbed = nn.Embedding(hyper.vs, hyper.hs)
        self.posit = PositionalEncoding(hyper.hs, hyper.cx)
        self.think = nn.TransformerEncoderLayer(d_model=hyper.hs, nhead=hyper.ah, dim_feedforward=hyper.hs*4, activation='gelu')
        self.thnkr = nn.TransformerEncoder(self.think, num_layers=hyper.ly)
        self.speak = nn.Linear(hyper.hs, hyper.vs)
        self.cmask= t.triu(t.ones(hyper.cx, hyper.cx) * float('-inf'), diagonal=1)
    def forward(self, x):
        x = self.inbed(x) * (hyper.hs ** .5)
        x = self.posit(x)
        x = self.thnkr(x, mask=self.cmask, is_causal=True)
        return self.speak(x)


In [14]:
# run = wandb.init(
#     project="tinystories",
#     config={
#         "learning_rate": hyper.lr,
#         "epochs": 10,
#     })

In [15]:
storytell = trans()
storytell = storytell.to(device)

print(f'There are {round((sum(p.numel() for p in storytell.parameters()) - hyper.vs*hyper.hs*2)/1e6, 1)} million parameters in the model, plus {round((hyper.vs*hyper.hs*2)/1e6, 1)} million embeddings parameters.')

There are 35.4 million parameters in the model, plus 12.6 million embeddings parameters.


In [16]:
# wandb.watch(storytell, log_freq=100)

In [17]:
optim = t.optim.Adam(storytell.parameters(), lr=hyper.lr)

In [18]:
lossf = nn.CrossEntropyLoss()

In [19]:
optim.load_state_dict(t.load('story_optim_0.pt', map_location=device))

In [20]:
storytell.load_state_dict(t.load('story_model_0.pt', map_location=device))

<All keys matched successfully>

In [ ]:
step = 0
for epoch in range(hyper.ep):
    for batch in trainl:
        seq = batch['input_ids'].to(device)
        out = storytell(seq)
        loss = lossf(t.flatten(out, end_dim=1), t.flatten(t.roll(seq, -1)))
        loss.backward()

        optim.step()
        optim.zero_grad()

        if step % 10 == 0:
            print(f'Step {step} of {len(trainl)}: loss {loss.item()}')

        if step % 100 == 0:
            t.save(storytell.state_dict(), f'story_model_{step}.pt')
            t.save(optim.state_dict(), f'story_optim_{step}.pt')
        
            with t.no_grad():
                tloss = 0
                steps = 0
                storytell.eval()
                for batch in validl:
                    seq = batch['input_ids'].to(device)
                    out = storytell(seq)
                    tloss += lossf(t.flatten(out, end_dim=1), t.flatten(t.roll(seq, -1))).item()
                    steps += 1
                print(f'validation: loss {tloss/steps}')
                storytell.train()
        step += 1

Step 0 of 9673: loss 1.7828541994094849
validation: loss 1.7633733073460687
Step 10 of 9673: loss 1.7870320081710815
Step 20 of 9673: loss 1.7119168043136597
Step 30 of 9673: loss 1.7623684406280518
Step 40 of 9673: loss 1.7257035970687866
Step 50 of 9673: loss 1.8138842582702637
Step 60 of 9673: loss 1.8084945678710938
Step 70 of 9673: loss 1.7809900045394897
Step 80 of 9673: loss 1.8043197393417358
Step 90 of 9673: loss 1.8057655096054077
Step 100 of 9673: loss 1.8252079486846924
validation: loss 1.7632240179887753
Step 110 of 9673: loss 1.76325523853302
Step 120 of 9673: loss 1.7604920864105225
Step 130 of 9673: loss 1.816753625869751
Step 140 of 9673: loss 1.8630592823028564
Step 150 of 9673: loss 1.772922396659851
Step 160 of 9673: loss 1.7300993204116821
Step 170 of 9673: loss 1.7685534954071045
Step 180 of 9673: loss 1.736989974975586
Step 190 of 9673: loss 1.7772109508514404
Step 200 of 9673: loss 1.836031198501587
validation: loss 1.7626174479415737
Step 210 of 9673: loss 1.75

validation: loss 1.7522091263348294
Step 11210 of 9673: loss 1.7781798839569092
Step 11220 of 9673: loss 1.7292336225509644
Step 11230 of 9673: loss 1.7755799293518066
Step 11240 of 9673: loss 1.7351405620574951
Step 11250 of 9673: loss 1.7968978881835938
Step 11260 of 9673: loss 1.8106435537338257
Step 11270 of 9673: loss 1.7161171436309814
Step 11280 of 9673: loss 1.7723462581634521
Step 11290 of 9673: loss 1.720647931098938
Step 11300 of 9673: loss 1.786023736000061
validation: loss 1.7517737690935429
Step 11310 of 9673: loss 1.7781120538711548
Step 11320 of 9673: loss 1.8072514533996582
Step 11330 of 9673: loss 1.8034768104553223
Step 11340 of 9673: loss 1.7251840829849243
Step 11350 of 9673: loss 1.6844674348831177
Step 11360 of 9673: loss 1.7952377796173096
Step 11370 of 9673: loss 1.6695829629898071
Step 11380 of 9673: loss 1.7350188493728638
Step 11390 of 9673: loss 1.792580008506775
Step 11400 of 9673: loss 1.7442033290863037
validation: loss 1.753219860116231
Step 11410 of 96

KeyboardInterrupt: 

In [22]:

# idx = random.randint(0, len(valid) - 1)
# print(idx)

# print(tok.decode(valid['input_ids'][idx]))

# print('model gen:')
# print(tok.decode(storytell(valid['input_ids'][idx].unsqueeze(0).to(t.long)).argmax(dim=-1)[0]))

In [19]:
def generate_text(prompt, model, tokenizer, temperature=1.0, max_len=512):
    model.eval()
    with t.no_grad():
        input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)[:, :-1]
        print(input_ids.shape)
        cur_len = input_ids.shape[1]
        while cur_len < max_len:
            outputs = model(input_ids)
            next_token_logits = outputs[0][-1, :] / temperature
            next_token_logits[1] = -float('inf')
            next_token_id = t.multinomial(t.softmax(next_token_logits, dim=-1), num_samples=1).unsqueeze(-1)
            input_ids = t.cat([input_ids, next_token_id], dim=1)
            cur_len += 1
            if next_token_id[0][0] == tokenizer.eos_token_id:
                break
        return tokenizer.decode(input_ids.squeeze()[55:], skip_special_tokens=False)
    
idx = random.randint(0, len(valid) - 1)
prompt = tok.decode(valid['input_ids'][idx][:60])
print(prompt)
print(generate_text(prompt, storytell, tok, temperature=1, max_len=200))

Features:  dialogue
Words:  watch, pasta, nosy
Story: 

 abby was so excited to go to the park. she was on her way when she noticed the pasta shop. she wanted to get a snack before the park.

 abby's mom told her to be careful with the money. abby
torch.Size([1, 60])
 with the money. abby ate it in the ladder was about him with water. one day, her mommy tried to the truck to balance. it. he encountered a little boy named timmy. as they saw a big pose! i can listen carefully this movie could, there was a big open the little seed felt good idea, "because i could fly and lay down. you and dance in the balls and bob loved to buy a big box. she found a boy, "the moon, the little girl was to the sun that she loved paper otter was so glad to play with a mean children and unique journey home and welcomed jack was told her tomatoes and kept flying high for the wet leaf! 
Story:  once upon
